In [1]:
import sys
import ROOT
from ROOT import gROOT
from ROOT import gStyle
%jsroot on

ROOT.gInterpreter.AddIncludePath('../../include/')
ROOT.gInterpreter.Declare('#include "RoEvent.h"')
ROOT.gSystem.Load('../../libs/libRoEvent.so')

gROOT.LoadMacro("../style/CMS/tdrstyle.C")
gROOT.ProcessLine("setTDRStyle();")
gROOT.LoadMacro("../style/CMS/CMS_lumi.C")

import numpy as np

#from array import array
from pathlib import Path
sys.path.append("../package/")
import lycro as lyplot

gStyle.SetOptFit(0)

Welcome to JupyROOT 6.24/00


In [2]:
run = "r1202"
root = "../../tmp/sig_" + run + ".root"
print("ROOT file: {}".format(root))
AMC = "000"
threshold = 90; window = 100; size = 100; chs = range(0,64); cards = [0, 0]

ROOT file: ../../tmp/sig_r1202.root


In [3]:
unusual = []

prof_pos = ROOT.TH2D("profile_pos", "profile_pos", window, 0, window, 4096, 0, 4096)
height_pos = ROOT.TH1D("height_pos", "height_pos", size, 1, -1); sum_pos = ROOT.TH1D("sum_pos", "sum_pos", size, 1, -1)

arr_height_pos = []; arr_sum_pos = []
for i in range(len(chs)):
    h_h_pos = ROOT.TH1D("height_pos_ch" + str(chs[i]), "height_pos_ch" + str(chs[i]), size, 1, -1); arr_sum_pos.append(h_h_pos)
    h_sum_pos = ROOT.TH1D("sum_pos_ch" + str(chs[i]), "sum_pos_ch" + str(chs[i]), size, 1, -1); arr_height_pos.append(h_sum_pos)
    
f = ROOT.TFile(root)
myTree = f.Get("data");
isig = 0; numSigs = myTree.GetEntries()
for entry in myTree:
    sig = entry.sigs
    peak = sig.peak
    pol = sig.pol
    ch = sig.ch
    card = sig.card
    if(pol == -1): continue 
    if(not(ch in chs and card in cards)): continue
    size = len(sig.signal)
    if(len(sig.pedestal) != size or size == 0): unusual.append(isig); continue
    #level = np.average(sig.pedestal); 
    level = 2048
    data = []   
    for i in range(size):
        tmp = sig.signal[i]
        prof_pos.Fill(i, tmp)
        data.append(pol*(tmp - level))
    #height = np.max(data); integral = np.sum(data)
    #height_pos.Fill(height); arr_height_pos[ch].Fill(height)
    #sum_pos.Fill(integral); arr_sum_pos[ch].Fill(integral)
    
    isig += 1
    if(isig%100 == 0 or isig == numSigs): 
        print("Sig {}:{} is processing...".format(isig, numSigs), end='\r'); 